## Тут ми проводимо клонування репозиторію та запуск colab_bootstap для того щоб підключити одразу все і встановити всі необхідні бібліотеки


In [ ]:
!git clone https://github.com/YuraKucher/rag-engine.git


Cloning into 'rag-engine'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (325/325), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 325 (delta 144), reused 264 (delta 87), pack-reused 0 (from 0)
Receiving objects: 100% (325/325), 113.18 KiB | 2.36 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [ ]:
%cd /content/rag-engine
!ls

/content/rag-engine
ARCHITECTURE.md  dev_rag_pipeline.ipynb  requirements.txt  tests
config		 notebooks		 schemas	   ui
core		 README.md		 services


In [ ]:
%run notebooks/colab_bootstrap.ipynb


✅ Running in Google Colab
Mounted at /content/drive
✅ Google Drive mounted at /content/drive
Project root: /content/rag-engine
Drive root: /content/drive/MyDrive
✅ Repository already exists
📁 Repo contents:
total 372
-rw-r--r--  1 root root   9906 Jan  9 10:50 ARCHITECTURE.md
drwxr-xr-x  2 root root   4096 Jan  9 10:50 config
drwxr-xr-x 13 root root   4096 Jan  9 10:50 core
-rw-r--r--  1 root root 333043 Jan  9 10:50 dev_rag_pipeline.ipynb
drwxr-xr-x  2 root root   4096 Jan  9 10:50 notebooks
-rw-r--r--  1 root root      0 Jan  9 10:50 README.md
-rw-r--r--  1 root root    922 Jan  9 10:50 requirements.txt
drwxr-xr-x  2 root root   4096 Jan  9 10:50 schemas
drwxr-xr-x  2 root root   4096 Jan  9 10:50 services
drwxr-xr-x  2 root root   4096 Jan  9 10:50 tests
drwxr-xr-x  3 root root   4096 Jan  9 10:50 ui
/content/rag-engine
📦 Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation

✅ Core imports successful
Python: 3.12.12
OS: Linux
Torch: 2.9.0+cpu
CUDA available: False


## Тут починається вже робота з пайплайном


### Bootstrap / paths

In [ ]:
from pathlib import Path
import sys

PROJECT_ROOT = Path("/content/rag-engine")
assert PROJECT_ROOT.exists(), "Project root not found"

sys.path.append(str(PROJECT_ROOT))

DATA_ROOT = Path("/content/drive/MyDrive/rag-data")
DATA_ROOT.mkdir(parents=True, exist_ok=True)

PATHS = {
    "documents": DATA_ROOT / "documents",
    "chunks": DATA_ROOT / "chunks",
    "indexes": DATA_ROOT / "indexes",
    "evaluations": DATA_ROOT / "evaluations",
    "feedback": DATA_ROOT / "feedback"
}

for p in PATHS.values():
    p.mkdir(parents=True, exist_ok=True)

print("✅ Paths ready")


✅ Paths ready


#### Перевірка наявності документів в documents

In [ ]:
list(PATHS["documents"].glob("*"))

[PosixPath('/content/drive/MyDrive/rag-data/documents/Daggerfall Unity Manual.pdf'),
 PosixPath('/content/drive/MyDrive/rag-data/documents/MakingofDaggerfall.pdf')]

### Ініціалізація RAGService

In [ ]:
from services.rag_service import RAGService

rag = RAGService(
    documents_path=str(PATHS["documents"]),
    chunks_path=str(PATHS["chunks"]),
    feedback_path=str(PATHS["feedback"]),
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    llm_model="llama2"  # або інша локальна модель
)

print("✅ RAGService initialized")


Device set to use cpu


✅ RAGService initialized


### Ingestion + indexing

In [ ]:
results = []

for pdf in PATHS["documents"].glob("*.pdf"):
    print(f"\n📄 Ingesting: {pdf.name}")

    result = rag.ingest_document(
        source=str(pdf),
        file_type="pdf"
    )

    results.append(result)

    print("→ chunks:", result["chunks_count"])
    print("→ index_id:", result["index"]["index_id"])

results



📄 Ingesting: Daggerfall Unity Manual.pdf
→ chunks: 30
→ index_id: da576ac5-76dd-49cd-878c-bfffe6b04a06

📄 Ingesting: MakingofDaggerfall.pdf
→ chunks: 40
→ index_id: afbb9010-1c06-46f0-b59a-d231395fbc2a


[{'document_id': '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4',
  'chunks_count': 30,
  'index': {'index_id': 'da576ac5-76dd-49cd-878c-bfffe6b04a06',
   'index_type': 'faiss',
   'embedding_model': 'sentence-transformers/all-MiniLM-L6-v2',
   'index_path': '/content/drive/MyDrive/rag-data/indexes/da576ac5-76dd-49cd-878c-bfffe6b04a06.faiss',
   'chunk_ids': ['4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_0',
    '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_1',
    '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_2',
    '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_3',
    '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_4',
    '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_5',
    '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_6',
    '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_7',
    '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_8',
    '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_9',
    '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_10',
    '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_11',
    '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_12',
    '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_13',
   

#### Перевірка, що індекси реально збережені

In [ ]:
list(PATHS["indexes"].glob("*"))

[PosixPath('/content/drive/MyDrive/rag-data/indexes/da576ac5-76dd-49cd-878c-bfffe6b04a06.faiss'),
 PosixPath('/content/drive/MyDrive/rag-data/indexes/da576ac5-76dd-49cd-878c-bfffe6b04a06.index.json'),
 PosixPath('/content/drive/MyDrive/rag-data/indexes/afbb9010-1c06-46f0-b59a-d231395fbc2a.faiss'),
 PosixPath('/content/drive/MyDrive/rag-data/indexes/afbb9010-1c06-46f0-b59a-d231395fbc2a.index.json')]

### Питання (warm run) / Відповідь

In [ ]:
question = "What is Daggerfall?"

response = rag.ask(question)

response


{'question': 'What is Daggerfall?',
 'answer': 'Obtain a working install of Daggerfall, patched to .213.',
 'evaluation': {'evaluation_id': '14fd885f-edc9-41a7-a1a6-b8e1efc103eb',
  'question': 'What is Daggerfall?',
  'answer': 'Obtain a working install of Daggerfall, patched to .213.',
  'index_id': 'da576ac5-76dd-49cd-878c-bfffe6b04a06',
  'chunk_ids': ['4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_0',
   '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_1',
   '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_5'],
  'metrics': {'relevance': 0.655, 'groundedness': 0.81, 'answerability': 0.6},
  'created_at': '2026-01-09T09:25:39.054291'},
 'sources': [{'chunk_id': '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_0',
   'document_id': '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4'},
  {'chunk_id': '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_1',
   'document_id': '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4'},
  {'chunk_id': '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_5',
   'document_id': '4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4'}],
 'feedback_id': '14fd8

### Контроль: Оцінка та Фідбек

In [ ]:
response["evaluation"]

{'evaluation_id': '1c713ec4-a36d-44d4-92ec-e0ee5eca5cf1',
 'question': 'What is Daggerfall?',
 'answer': 'Bethesda showed a trailer for Daggerfall at Winter CES in January 1995.',
 'index_id': 'afbb9010-1c06-46f0-b59a-d231395fbc2a',
 'chunk_ids': ['064ecbef-d6a1-493a-a5d1-d3a01f23bdf5_7',
  '064ecbef-d6a1-493a-a5d1-d3a01f23bdf5_8',
  '064ecbef-d6a1-493a-a5d1-d3a01f23bdf5_20'],
 'metrics': {'relevance': 0.577, 'groundedness': 0.833, 'answerability': 0.6},
 'created_at': '2026-01-09T09:21:08.698527'}

In [ ]:
response["feedback_id"]

'1c713ec4-a36d-44d4-92ec-e0ee5eca5cf1'

## Після рестарту

In [ ]:
from pathlib import Path
import sys

PROJECT_ROOT = Path("/content/rag-engine")
sys.path.append(str(PROJECT_ROOT))

DATA_ROOT = Path("/content/drive/MyDrive/rag-data")

PATHS = {
    "documents": DATA_ROOT / "documents",
    "chunks": DATA_ROOT / "chunks",
    "indexes": DATA_ROOT / "indexes",
    "evaluations": DATA_ROOT / "evaluations",
    "feedback": DATA_ROOT / "feedback"
}

print("✅ Cold start paths loaded")


✅ Cold start paths loaded


In [ ]:
from services.rag_service import RAGService

rag = RAGService(
    documents_path=str(PATHS["documents"]),
    chunks_path=str(PATHS["chunks"]),
    feedback_path=str(PATHS["feedback"]),
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    llm_model="llama2"
)

print("✅ RAGService re-initialized")


/content/rag-engine/core/indexing/embedder.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  self._embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
/content/rag-engine/core/generation/llm_client.py:47: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  self.llm: LLM = HuggingFacePipeline(


✅ RAGService re-initialized


In [ ]:
from core.indexing.index_manager import IndexManager

index_files = list(PATHS["indexes"].glob("*.index.json"))
assert index_files, "No index metadata found"

index_id = index_files[0].stem.replace(".index", "")

rag.index_manager.load_index(index_id)

print("✅ Index loaded:", index_id)
print("Chunks in index:", len(rag.index_manager.chunk_ids))


Index loaded: da576ac5-76dd-49cd-878c-bfffe6b04a06
Chunks in index: 30


In [ ]:
question = "Tell me about Daggerfall Unity"
response_cold = rag.ask(question)

response_cold


{'question': 'What is Daggerfall Unity?',
 'answer': 'I do not know.',
 'evaluation': {'evaluation_id': '9530f970-e3b4-4b3d-9cd2-dd86cf4e3a2a',
  'question': 'What is Daggerfall Unity?',
  'answer': 'I do not know.',
  'index_id': 'afbb9010-1c06-46f0-b59a-d231395fbc2a',
  'chunk_ids': ['064ecbef-d6a1-493a-a5d1-d3a01f23bdf5_7',
   '064ecbef-d6a1-493a-a5d1-d3a01f23bdf5_20',
   '064ecbef-d6a1-493a-a5d1-d3a01f23bdf5_13'],
  'metrics': {'relevance': 1.0, 'groundedness': 1.0, 'answerability': 0.0},
  'created_at': '2026-01-09T10:56:20.947598'},
 'sources': [{'chunk_id': '064ecbef-d6a1-493a-a5d1-d3a01f23bdf5_7',
   'document_id': '064ecbef-d6a1-493a-a5d1-d3a01f23bdf5'},
  {'chunk_id': '064ecbef-d6a1-493a-a5d1-d3a01f23bdf5_20',
   'document_id': '064ecbef-d6a1-493a-a5d1-d3a01f23bdf5'},
  {'chunk_id': '064ecbef-d6a1-493a-a5d1-d3a01f23bdf5_13',
   'document_id': '064ecbef-d6a1-493a-a5d1-d3a01f23bdf5'}],
 'feedback_id': '9530f970-e3b4-4b3d-9cd2-dd86cf4e3a2a'}

In [ ]:
print("Warm answer:\n", response["answer"][:500])
print("\nCold answer:\n", response_cold["answer"][:500])


Warm answer:
 Obtain a working install of Daggerfall, patched to .213.

Cold answer:
 4dd11d-a1ec-41ac-a2cd-33bbc817f7e4, chunk: 4e4dd11d-a1ec-41ac-a2cd-33bbc817f7e4_0)
